In [1]:
import pandas as pd
import numpy as np
import time
import folium   
from folium.plugins import MarkerCluster


/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/sensor_meta_feature.csv' , sep ="\t")


In [37]:
df2 = pd.read_csv("/Users/bugragorkem/Desktop/try1.csv", sep ="\t")

In [25]:
df

,Unnamed: 0.1,Unnamed: 0,Fwy,District,County,City,Abs PM,Length,station_id,Name,...,Inner Median Type,Inner Median Width,Terrain,Population,Barrier,Surface,Roadway Use,Direction,Lat,Lng
0,0,0,SR1-N,12,Orange,Newport Beach,12.401,2.695,1221176,REEF POINT DR,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.565462,-117.828018
1,1,1,SR1-N,12,Orange,Newport Beach,12.791,0.485,1221170,CRYSTAL HEIGHTS,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.568992,-117.833412
2,2,2,SR1-N,12,Orange,Newport Beach,13.371,1.825,1221163,1 NB LOS TRANCOS,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.575502,-117.839807
3,3,3,SR1-N,12,Orange,Newport Beach,16.441,1.825,1221148,1 NB AVOCADO AVE,...,Paved - Roadway Use,14 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.605811,-117.875914
4,4,4,SR1-N,12,Orange,Newport Beach,17.021,0.595,1221143,1 NB IRVINE TERRACE,...,Paved - Roadway Use,14 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.611026,-117.883525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16967,16967,16967,I980-W,4,Alameda,Oakland,0.971,NaN,408594,18th St off,...,Separate Structure,50 ft,Flat,Urbanized,Concrete Barrier,"Base & Surface >= 7"" Thick",No Special Features,W,37.810554,-122.275579
16968,16968,16968,I980-W,4,Alameda,Oakland,0.971,0.305,401610,14th/18th St,...,Separate Structure,50 ft,Flat,Urbanized,Concrete Barrier,"Base & Surface >= 7"" Thick",No Special Features,W,37.810554,-122.275579
16969,16969,16969,I980-W,4,Alameda,Oakland,1.301,0.320,401416,23rd St,...,Paved - No Roadway Use,40 ft,Flat,Urbanized,Metal Beam Barrier,"Base & Surface >= 7"" Thick",Auxiliary Lane (Inc in No Lanes Fields),W,37.814242,-122.271837
16970,16970,16970,I980-W,4,Alameda,Oakland,1.611,NaN,401912,29th street,...,Separate Grades,80 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",Auxiliary Lane (Inc in No Lanes Fields),W,37.818437,-122.270170


In [11]:
nan_analysis = df.isna().sum().to_frame(name='NaN_Anzahl')
nan_analysis['NaN_Prozent'] = (df.isna().mean() * 100).round(2)


print(nan_analysis)


                      NaN_Anzahl  NaN_Prozent
Unnamed: 0.1                   0         0.00
Unnamed: 0                     0         0.00
Fwy                            0         0.00
District                       0         0.00
County                         0         0.00
City                        1891        11.14
Abs PM                         0         0.00
Length                      6354        37.44
station_id                     0         0.00
Name                           7         0.04
Type                           0         0.00
Sensor Type                 4064        23.95
HOV                            0         0.00
Road Width                   833         4.91
Lane Width                   833         4.91
Inner Shoulder Width         833         4.91
Outer Shoulder Width         833         4.91
Design Speed Limit           833         4.91
Inner Median Type            833         4.91
Inner Median Width           833         4.91
Terrain                      833  

In [27]:
# Anzahl und Prozentsatz der fehlenden Werte in 'Length' nach 'Type' gruppiert
missing_length_by_type = df.groupby('Type')['Length'].apply(lambda x: x.isna().sum())
total_count_by_type = df.groupby('Type')['Length'].size()

# DataFrame mit der Anzahl und dem Prozentsatz der fehlenden Werte in 'Length' für jede 'Type'-Gruppe
missing_length_analysis = pd.DataFrame({
    'fehlende Daten': missing_length_by_type,
    #'Total_Count': total_count_by_type,
    'Fehlende Daten in Prozent': (missing_length_by_type / total_count_by_type * 100).round(2)
})

print(missing_length_analysis)

                      fehlende Daten  Fehlende Daten in Prozent
Type                                                           
Coll/Dist                         49                     100.00
Conventional Highway             101                     100.00
Fwy-Fwy                          465                     100.00
HOV                                0                       0.00
Mainline                           0                       0.00
Off Ramp                        2392                     100.00
On Ramp                         3347                      99.97


In [28]:
on_ramp = df[(df['Type'] == 'On Ramp') & (df['Length'].notna())]
on_ramp

,Unnamed: 0.1,Unnamed: 0,Fwy,District,County,City,Abs PM,Length,station_id,Name,...,Inner Median Type,Inner Median Width,Terrain,Population,Barrier,Surface,Roadway Use,Direction,Lat,Lng
15397,15397,15397,I580-W,4,Alameda,Livermore,25.994,0.22,416141,Vasco Rd on diag,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,37.708304,-121.725887


In [17]:
df = df.drop(columns = ['HOV'])

In [18]:
df

,Fwy,District,County,City,Abs PM,Length,station_id,Name,Type,Sensor Type,...,Inner Median Type,Inner Median Width,Terrain,Population,Barrier,Surface,Roadway Use,Lat,Lng,Direction
0,SR1-N,12,Orange,NaN,12.401,2.695,1221176,REEF POINT DR,Mainline,NaN,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,33.565462,-117.828018,N
1,SR1-N,12,Orange,NaN,12.791,0.485,1221170,CRYSTAL HEIGHTS,Mainline,NaN,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,33.568992,-117.833412,N
2,SR1-N,12,Orange,NaN,13.371,1.825,1221163,1 NB LOS TRANCOS,Mainline,NaN,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,33.575502,-117.839807,N
3,SR1-N,12,Orange,Newport Beach,16.441,1.825,1221148,1 NB AVOCADO AVE,Mainline,NaN,...,Paved - Roadway Use,14 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,33.605811,-117.875914,N
4,SR1-N,12,Orange,Newport Beach,17.021,0.595,1221143,1 NB IRVINE TERRACE,Mainline,NaN,...,Paved - Roadway Use,14 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,33.611026,-117.883525,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16967,I980-W,4,Alameda,Oakland,0.971,NaN,408594,18th St off,Off Ramp,loops,...,Separate Structure,50 ft,Flat,Urbanized,Concrete Barrier,"Base & Surface >= 7"" Thick",No Special Features,37.810554,-122.275579,W
16968,I980-W,4,Alameda,Oakland,0.971,0.305,401610,14th/18th St,Mainline,loops,...,Separate Structure,50 ft,Flat,Urbanized,Concrete Barrier,"Base & Surface >= 7"" Thick",No Special Features,37.810554,-122.275579,W
16969,I980-W,4,Alameda,Oakland,1.301,0.320,401416,23rd St,Mainline,loops,...,Paved - No Roadway Use,40 ft,Flat,Urbanized,Metal Beam Barrier,"Base & Surface >= 7"" Thick",Auxiliary Lane (Inc in No Lanes Fields),37.814242,-122.271837,W
16970,I980-W,4,Alameda,Oakland,1.611,NaN,401912,29th street,Fwy-Fwy,loops,...,Separate Grades,80 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",Auxiliary Lane (Inc in No Lanes Fields),37.818437,-122.270170,W


In [29]:
nan_city = df[df["City"].isna()]

In [30]:
# Erstelle eine zentrale Karte von Kalifornien
kalifornien_karte = folium.Map(location=[37.5, -119.5], zoom_start=6)

# Füge Marker für jeden Punkt mit NaN-Wert in "City" hinzu
for idx, row in nan_city.iterrows():
    lat, lng = row['Lat'], row['Lng']
    if pd.notna(lat) and pd.notna(lng):  # Stelle sicher, dass Lat/Lng-Werte existieren
        folium.Marker(
            location=[lat, lng],
            popup=f"NaN City at index: {idx}",
            icon=folium.Icon(color="red", icon="exclamation")
        ).add_to(kalifornien_karte)



In [14]:
# Erstelle eine zentrale Karte von Kalifornien
kalifornien_karte = folium.Map(location=[37.5, -119.5], zoom_start=6)

# Erstelle ein Marker-Cluster-Objekt
marker_cluster = MarkerCluster().add_to(kalifornien_karte)

# Füge die Punkte zum Cluster hinzu
for idx, row in nan_city.iterrows():
    lat, lng = row['Lat'], row['Lng']
    if pd.notna(lat) and pd.notna(lng):  # Stelle sicher, dass Lat/Lng-Werte existieren
        folium.Marker(
            location=[lat, lng],
            popup=f"NaN City at index: {idx}",
            icon=folium.Icon(color="red", icon="exclamation")
        ).add_to(marker_cluster)


In [31]:
kalifornien_karte

In [59]:
missing_data = df[(df['Lat'].isna())]

In [60]:
missing_data

,Fwy,District,County,City,Abs PM,Length,station_id,Name,Type,Sensor Type,...,Inner Median Type,Inner Median Width,Terrain,Population,Barrier,Surface,Roadway Use,Lat,Lng,Direction


In [58]:
df.at[12566 , 'Lat'] = 38.604135
df.at[12566, 'Lng'] =  -121.434784

In [15]:
test4 = pd.read_csv('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/try1.csv', sep ="\t")

In [16]:
nan_analysis = test4.isna().sum().to_frame(name='NaN_Count')
nan_analysis['NaN_Percentage'] = (df.isna().mean() * 100).round(2)
print(nan_analysis)

                      NaN_Count  NaN_Percentage
Unnamed: 0                    0            0.00
Fwy                           0            0.00
District                      0            0.00
County                        0            0.00
City                       2250           29.50
Abs PM                        0            0.00
Length                     6354           37.44
station_id                    0            0.00
Name                          7            0.04
Type                          0            0.00
Sensor Type                4064           23.95
HOV                           0            0.00
Road Width                  833            4.91
Lane Width                  833            4.91
Inner Shoulder Width        833            4.91
Outer Shoulder Width        833            4.91
Design Speed Limit          833            4.91
Inner Median Type           833            4.91
Inner Median Width          833            4.91
Terrain                     833         

In [32]:
from geopy import Nominatim

# Filtern der Zeilen, bei denen die Stadt fehlt, aber Lat und Lng vorhanden sind
missing_cities = df[df['City'].isna() & df['Lat'].notna() & df['Lng'].notna()]

# Geolocator initialisieren
geolocator = Nominatim(user_agent="city_filler")

In [33]:
# Funktion zur Abfrage der Stadt basierend auf Lat und Lng
def get_city(lat, lng):
    try:
        location = geolocator.reverse((lat, lng), exactly_one=True)
        if location and 'city' in location.raw['address']:
            return location.raw['address']['city']
        elif location and 'town' in location.raw['address']:
            return location.raw['address']['town']
        elif location and 'village' in location.raw['address']:
            return location.raw['address']['village']
    except Exception as e:
        print(f"Fehler bei der Abfrage für ({lat}, {lng}): {e}")
    return None

In [34]:
from tqdm import tqdm

# Fortschrittsanzeige
tqdm.pandas()

# Fehlende Städte ergänzen
df.loc[missing_cities.index, 'City'] = missing_cities.progress_apply(
    lambda row: get_city(row['Lat'], row['Lng']), axis=1
)

# Ergebnisse anzeigen
print(df)

100%|██████████| 1891/1891 [17:00<00:00,  1.85it/s]

       Unnamed: 0.1  Unnamed: 0     Fwy  District   County           City  \
0                 0           0   SR1-N        12   Orange  Newport Beach   
1                 1           1   SR1-N        12   Orange  Newport Beach   
2                 2           2   SR1-N        12   Orange  Newport Beach   
3                 3           3   SR1-N        12   Orange  Newport Beach   
4                 4           4   SR1-N        12   Orange  Newport Beach   
...             ...         ...     ...       ...      ...            ...   
16967         16967       16967  I980-W         4  Alameda        Oakland   
16968         16968       16968  I980-W         4  Alameda        Oakland   
16969         16969       16969  I980-W         4  Alameda        Oakland   
16970         16970       16970  I980-W         4  Alameda        Oakland   
16971         16971       16971  I980-W         4  Alameda        Oakland   

       Abs PM  Length  station_id                 Name  ...  \
0      12.40

In [11]:
# Zeilen filtern, bei denen die City fehlt, aber Lat und Lng vorhanden sind
missing_cities = df[df['City'].isna() & df['Lat'].notna() & df['Lng'].notna()]

In [12]:
#Geolocator initialisieren
geolocator = Nominatim(user_agent="city_finder")

In [13]:
def get_city(lat, lng):
    try:
        # Reverse Geocoding für die gegebene Koordinate
        location = geolocator.reverse((lat, lng), exactly_one=True)
        
        if location:
            address = location.raw.get('address', {})
            
            # Direkt nach einer Stadt suchen
            if 'city' in address:
                return address['city']
            elif 'town' in address:
                return address['town']
            elif 'village' in address:
                return address['village']
            elif 'suburb' in address:
                # Wenn ein Stadtteil vorhanden ist, überprüfe die Nachbarschaft
                return address.get('neighbourhood', address.get('suburb'))

        # Falls keine Stadtinformationen gefunden werden
        # Suche die nächstgelegene Stadt
        location = geolocator.reverse((lat, lng), exactly_one=False)
        for loc in location:
            address = loc.raw.get('address', {})
            if 'city' in address or 'town' in address or 'village' in address:
                return address.get('city') or address.get('town') or address.get('village')

    except Exception as e:
        print(f"Fehler bei der Abfrage für ({lat}, {lng}): {e}")

    # Rückgabe None, falls keine Stadt gefunden werden kann
    return None

In [14]:
city_test = get_city(36.667246, -121.814121)

In [15]:
print(city_test)

Marina


In [16]:
# Fortschrittsanzeige
tqdm.pandas()

# Fehlende Städte ergänzen
df.loc[missing_cities.index, 'City'] = missing_cities.progress_apply(
    lambda row: get_city(row['Lat'], row['Lng']), axis=1
)

 41%|████▏     | 933/2250 [14:13<51:35,  2.35s/it]

Fehler bei der Abfrage für (38.042178, -122.242478): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.042178&lon=-122.242478&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x12b198a30>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))


100%|██████████| 2250/2250 [34:56<00:00,  1.07it/s]


In [17]:
df

,Unnamed: 0,Fwy,District,County,City,Abs PM,Length,station_id,Name,Type,...,Inner Median Type,Inner Median Width,Terrain,Population,Barrier,Surface,Roadway Use,Direction,Lat,Lng
0,0,SR1-N,12,Orange,Newport Beach,12.401,2.695,1221176,REEF POINT DR,Mainline,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.565462,-117.828018
1,1,SR1-N,12,Orange,Newport Beach,12.791,0.485,1221170,CRYSTAL HEIGHTS,Mainline,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.568992,-117.833412
2,2,SR1-N,12,Orange,Newport Beach,13.371,1.825,1221163,1 NB LOS TRANCOS,Mainline,...,Paved - No Roadway Use,12 ft,Rolling,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.575502,-117.839807
3,3,SR1-N,12,Orange,Newport Beach,16.441,1.825,1221148,1 NB AVOCADO AVE,Mainline,...,Paved - Roadway Use,14 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.605811,-117.875914
4,4,SR1-N,12,Orange,Newport Beach,17.021,0.595,1221143,1 NB IRVINE TERRACE,Mainline,...,Paved - Roadway Use,14 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",No Special Features,N,33.611026,-117.883525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16967,16967,I980-W,4,Alameda,Oakland,0.971,NaN,408594,18th St off,Off Ramp,...,Separate Structure,50 ft,Flat,Urbanized,Concrete Barrier,"Base & Surface >= 7"" Thick",No Special Features,W,37.810554,-122.275579
16968,16968,I980-W,4,Alameda,Oakland,0.971,0.305,401610,14th/18th St,Mainline,...,Separate Structure,50 ft,Flat,Urbanized,Concrete Barrier,"Base & Surface >= 7"" Thick",No Special Features,W,37.810554,-122.275579
16969,16969,I980-W,4,Alameda,Oakland,1.301,0.320,401416,23rd St,Mainline,...,Paved - No Roadway Use,40 ft,Flat,Urbanized,Metal Beam Barrier,"Base & Surface >= 7"" Thick",Auxiliary Lane (Inc in No Lanes Fields),W,37.814242,-122.271837
16970,16970,I980-W,4,Alameda,Oakland,1.611,NaN,401912,29th street,Fwy-Fwy,...,Separate Grades,80 ft,Flat,Urbanized,No Barriers,"Base & Surface >= 7"" Thick",Auxiliary Lane (Inc in No Lanes Fields),W,37.818437,-122.270170


In [23]:
df.to_csv('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/sensor_meta_feature.csv' , sep ="\t")

In [20]:
# Erstelle eine zentrale Karte von Kalifornien
kalifornien_karte = folium.Map(location=[37.5, -119.5], zoom_start=6)

# Füge Marker für jeden Punkt mit NaN-Wert in "City" hinzu
for idx, row in nan_city.iterrows():
    lat, lng = row['Lat'], row['Lng']
    if pd.notna(lat) and pd.notna(lng):  # Stelle sicher, dass Lat/Lng-Werte existieren
        folium.Marker(
            location=[lat, lng],
            popup=f"NaN City at index: {idx}",
            icon=folium.Icon(color="red", icon="exclamation")
        ).add_to(kalifornien_karte)

# Erstelle eine zentrale Karte von Kalifornien
kalifornien_karte = folium.Map(location=[37.5, -119.5], zoom_start=6)

# Erstelle ein Marker-Cluster-Objekt
marker_cluster = MarkerCluster().add_to(kalifornien_karte)

# Füge die Punkte zum Cluster hinzu
for idx, row in nan_city.iterrows():
    lat, lng = row['Lat'], row['Lng']
    if pd.notna(lat) and pd.notna(lng):  # Stelle sicher, dass Lat/Lng-Werte existieren
        folium.Marker(
            location=[lat, lng],
            popup=f"NaN City at index: {idx}",
            icon=folium.Icon(color="red", icon="exclamation")
        ).add_to(marker_cluster)

In [21]:
kalifornien_karte = folium.Map(location=[37.5, -119.5], zoom_start=6)

In [22]:
kalifornien_karte

In [41]:
bin(55)

'0b110111'

In [17]:
def binär_zu_dezimal(binär):
    dezimal = 0
    for i, bit in enumerate(reversed(binär)):
        dezimal += int(bit) * (2 ** i)
    return dezimal

binär_zu_dezimal("")


11

In [38]:
bin(63)

'0b111111'